# Pip Install

In [2]:
!pip install boto3 astropy sfdmap progressbar2 GPUtil parse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 446.7 kB/s eta 0:00:0000:01
  Using cached astropy-5.2.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (11.2 MB)
  Using cached sfdmap-0.1.1-py3-none-any.whl
  Using cached progressbar2-4.2.0-py2.py3-none-any.whl (27 kB)
  Using cached GPUtil-1.4.0-py3-none-any.whl
  Using cached parse-1.19.0-py3-none-any.whl
  Using cached s3transfer-0.6.0-py3-none-any.whl (79 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 30.5 MB/s eta 0:00:0000:0100:01
  Using cached pyerfa-2.0.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (741 kB)
  Using cached python_utils-3.5.2-py2.py3-none-any.whl (24 kB)


# Initialization

In [3]:
# imports
import pandas as pd
import numpy as np
import os
import sys
import pickle
import boto3
from matplotlib import pyplot as plt
import time

# random seed
seed = 42
np.random.seed(seed)

# local files paths
local_home_dir_path = os.path.expanduser("~")
local_work_dir_path = os.path.join(local_home_dir_path, 'thesis2')
local_code_dir_path = os.path.join(local_work_dir_path , 'code')

# S3 file paths
endpoint_url = 'https://s3-west.nrp-nautilus.io'
bucket_name = 'tau-astro'
prefix = 'almogh'
s3_work_dir_path = os.path.join(prefix, 'thesis2')
s3_data_dir_path = os.path.join(s3_work_dir_path , 'data')
s3_models_dir_path = os.path.join(s3_work_dir_path , 'models')
s3_final_table_csv_path = os.path.join(s3_data_dir_path, 'SDSS_DR16_all.csv')

s3_client = boto3.client("s3", endpoint_url=endpoint_url)

# adding code folder to path
sys.path.insert(1, local_code_dir_path)
from s3 import to_s3_npy, to_s3_pkl, from_s3_npy, from_s3_pkl, to_s3_fig, from_s3_pickle

# Load RFs

In [4]:
# Load SmallRF
SmallRF = from_s3_pkl(s3_client = s3_client,
                      bucket_name = bucket_name,
                      path_in_bucket = 'almogh/thesis2/models/SmallRF/train/rf.pkl')

loading from uri: s3://tau-astro/almogh/thesis2/models/SmallRF/train/rf.pkl


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
# Load BigRF
BigRF = from_s3_pkl(s3_client = s3_client,
                      bucket_name = bucket_name,
                      path_in_bucket = 'almogh/thesis2/models/BigRF/train/rf.pkl')

loading from uri: s3://tau-astro/almogh/thesis2/models/BigRF/train/rf.pkl


# Loading distance matrices

In [6]:
s3_inference_dir_path_in_bucket = 'almogh/thesis2/eval/inference'
D_NN = from_s3_npy(s3_client = s3_client, bucket_name = bucket_name,
    path_in_bucket = os.path.join(s3_inference_dir_path_in_bucket,'D_NN2.npy'))
D_BigRF = from_s3_npy(s3_client = s3_client, bucket_name = bucket_name,
    path_in_bucket = os.path.join(s3_inference_dir_path_in_bucket,'D_BigRF.npy'))
D_SmallRF = from_s3_npy(s3_client = s3_client, bucket_name = bucket_name,
    path_in_bucket = os.path.join(s3_inference_dir_path_in_bucket,'D_SmallRF.npy'))

loading from uri: s3://tau-astro/almogh/thesis2/eval/inference/D_NN2.npy
loading from uri: s3://tau-astro/almogh/thesis2/eval/inference/D_BigRF.npy
loading from uri: s3://tau-astro/almogh/thesis2/eval/inference/D_SmallRF.npy


# Load data

In [7]:
# loading outliers
data_dir = 'almogh/thesis2/data/BigRF/train'
X = from_s3_npy(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(data_dir, 'spec.npy'))
gs = from_s3_pickle(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(data_dir, 'gs.pkl'))

loading from uri: s3://tau-astro/almogh/thesis2/data/BigRF/train/spec.npy
loading from uri: s3://tau-astro/almogh/thesis2/data/BigRF/train/gs.pkl


In [8]:
# loading outliers
outliers_data_dir = 'almogh/thesis2/data/Outliers'
X_outliers = from_s3_npy(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(outliers_data_dir, 'spec.npy'))
gs_outliers = from_s3_pickle(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = os.path.join(outliers_data_dir, 'gs.pkl'))

loading from uri: s3://tau-astro/almogh/thesis2/data/Outliers/spec.npy
loading from uri: s3://tau-astro/almogh/thesis2/data/Outliers/gs.pkl


In [9]:
# filtering low snr
snr_th = 3
I = gs_outliers.snMedian>snr_th
print('Filtering {0}/{1} outliers with SNR lower than {2}.'.format(len(gs_outliers)-sum(I),len(gs_outliers),snr_th))
X_outliers = X_outliers[I]
gs_outliers = gs_outliers[I]

Filtering 2/114 outliers with SNR lower than 3.


# Infer

## Calculate for the dataset

In [20]:
leafs = np.array(BigRF.apply(X), dtype=np.int16)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.5s finished


In [22]:
from joblib import Parallel, delayed
from uRF_SDSS import realLeaf_i
realLeafs = Parallel(n_jobs=-1, verbose=1)(delayed(realLeaf_i)(BigRF.estimators_[i].tree_.value, leafs[:, i]) for i in range(len(BigRF.estimators_)))
realLeafs = np.array(realLeafs, dtype=bool).T

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.8s finished


## Calculate for outliers

In [26]:
SNRs = np.array([2,4,6,8,10,15,20])

In [23]:
n_noisy_inst = 4

In [24]:
def dist_i_mp_outliers(i):
    
    mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
    mutRealLeafs = np.logical_and(outliers_realLeafs[i], realLeafs, dtype=np.int16).sum(1, dtype=np.float64)

    return np.divide(mutLeafs, mutRealLeafs, dtype=np.float16)

In [ ]:
D_BigRF_outliers = np.zeros(shape=(len(gs_outliers), len(SNRs), X.shape[0]))
for i in range(len(gs_outliers)):
    print('outlier {0}/{1}'.format(i+1,len(gs_outliers)))
    x_i = X_outliers[i].reshape(1,-1)
    std_sqrd_i = np.std(x_i)**2
    for i_snr,snr in enumerate(SNRs):
        print('snr={0}, {1}/{2}'.format(snr, i_snr+1, len(SNRs)))
        
        # preparing a noisy batch of the outlier for this snr
        N_std_x = np.sqrt(std_sqrd_i/snr)
        x_noisy_inst = np.repeat(x_i, n_noisy_inst, axis=0)
        x_noisy_inst += N_std_x*np.random.randn(x_noisy_inst.shape[0], x_noisy_inst.shape[1])

        # infer
        outliers_leafs = np.array(BigRF.apply(x_noisy_inst), dtype=np.int16)
        outliers_realLeafs = Parallel(n_jobs=-1, verbose=1)(delayed(realLeaf_i)(BigRF.estimators_[i].tree_.value, outliers_leafs[:, i]) for i in range(len(BigRF.estimators_)))
        outliers_realLeafs = np.array(outliers_realLeafs, dtype=bool).T
        D_BigRF_outliers[i,i_snr,:] = np.mean(np.concatenate([dist_i_mp_outliers(j).reshape(1,-1) for j in range(n_noisy_inst)], axis=0), axis=0)

outlier 1/112
snr=2, 1/7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 437 out of 500 | elapsed:    1.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.1s finished
/tmp/ipykernel_77/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_77/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool)

snr=4, 2/7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 437 out of 500 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.5s finished
/tmp/ipykernel_77/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_77/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool)

snr=6, 3/7


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished
/tmp/ipykernel_77/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_77/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutRealLeafs = np.logical_and(outlier

snr=20, 7/7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished
/tmp/ipykernel_77/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_77/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutRealLeafs = np.logical_and(outlier

outlier 86/112
snr=2, 1/7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished
/tmp/ipykernel_77/4056456762.py:3: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutLeafs = np.logical_and((leafs == outliers_leafs[i]), realLeafs, dtype=np.int16).sum(1)
/tmp/ipykernel_77/4056456762.py:4: DeprecationWarning: using `dtype=` in comparisons is only useful for `dtype=object` (and will do nothing for bool). This operation will fail in the future.
  mutRealLeafs = np.logical_and(outlier

In [ ]:
to_s3_npy(D_NN_outliers, s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = 'almogh/thesis2/eval/D_NN2_outliers.npy')